<a href="https://colab.research.google.com/github/kdwaaa/a/blob/main/%E7%AC%AC3%E6%9C%9F%E5%80%8B%E4%BA%BA%E8%A3%BD%E4%BD%9C_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np


class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3))  # 3x3の盤面を初期化
        self.current_player = 1  # 黒(1)が先行

    def reset(self):
        self.board = np.zeros((3, 3))  # ゲームのリセット
        self.current_player = 1
        return self.board

    def available_actions(self):
        return np.argwhere(self.board == 0)  # 石を置ける空きマスを返す

    def step(self, action):
        x, y = action
        if self.board[x, y] != 0:
            raise ValueError("その場所には置けません")
        self.board[x, y] = self.current_player

        reward = self.check_winner()
        done = reward != 0 or len(self.available_actions()) == 0

        if not done:
            self.current_player = -self.current_player  # プレイヤー交代

        return self.board, reward, done

    def check_winner(self):
        for i in range(3):
            # 行や列が同じ石で埋まっているか確認
            if np.all(self.board[i, :] == self.current_player) or np.all(self.board[:, i] == self.current_player):
                return self.current_player
        # 斜めの確認
        if np.all(np.diag(self.board) == self.current_player) or np.all(np.diag(np.fliplr(self.board)) == self.current_player):
            return self.current_player
        return 0


In [ ]:
import random

class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = {}  # Qテーブルを辞書で管理
        self.alpha = alpha  # 学習率
        self.gamma = gamma  # 割引率
        self.epsilon = epsilon  # ε-greedy法の探索率

    def get_q_value(self, state, action):
        return self.q_table.get((str(state), action), 0.0)

    def choose_action(self, state, available_actions):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(available_actions)  # 探索
        else:
            # Q値が最大の行動を選択
            q_values = [self.get_q_value(state, tuple(action)) for action in available_actions]
            max_q_value = max(q_values)
            max_actions = [action for action, q_value in zip(available_actions, q_values) if q_value == max_q_value]
            return random.choice(max_actions)  # 同じQ値の場合はランダム選択

    def update_q_value(self, state, action, reward, next_state, available_actions):
        best_next_q_value = max([self.get_q_value(next_state, tuple(next_action)) for next_action in available_actions], default=0.0)
        old_q_value = self.get_q_value(state, tuple(action))
        new_q_value = old_q_value + self.alpha * (reward + self.gamma * best_next_q_value - old_q_value)
        self.q_table[(str(state), tuple(action))] = new_q_value


In [ ]:
def train_agent(episodes=10000):
    env = TicTacToe()
    agent = QLearningAgent()
    for episode in range(episodes):
        state = env.reset()
        done = False
        while not done:
            available_actions = env.available_actions()
            action = agent.choose_action(state, available_actions)
            next_state, reward, done = env.step(action)

            if not done:
                available_actions = env.available_actions()
                agent.update_q_value(state, action, reward, next_state, available_actions)

            state = next_state

        if episode % 1000 == 0:
            print(f"エピソード {episode} 終了")

    print("学習完了")
    return agent


In [ ]:
import numpy as np
import random

class TicTacToe:
    def __init__(self):
        self.state = np.zeros((3, 3), dtype=int)  # 0: 空, 1: 黒, 2: 白
        self.current_player = 1  # 1: 黒（エージェント）, 2: 白（プレイヤー）

    def reset(self):
        self.state = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        return self.state

    def available_actions(self):
        """現在の空いているマスをリストで返す"""
        return [(i, j) for i in range(3) for j in range(3) if self.state[i, j] == 0]

    def step(self, action):
        """指定されたアクションを実行"""
        x, y = action
        if self.state[x, y] != 0:
            raise ValueError("無効な手です")

        self.state[x, y] = self.current_player
        done, winner = self.check_winner()

        reward = 0
        if done:
            if winner == 1:  # エージェント（黒）が勝ち
                reward = 1
            elif winner == 2:  # プレイヤー（白）が勝ち
                reward = -1
            # 引き分けの場合は reward = 0 のまま

        self.current_player = 3 - self.current_player  # プレイヤー交代
        return self.state, reward, done

    def check_winner(self):
        """ゲーム終了状態かどうかを判定"""
        # 各行、列、対角線で3つ並んでいるか確認
        for i in range(3):
            if np.all(self.state[i, :] == self.current_player):
                return True, self.current_player
            if np.all(self.state[:, i] == self.current_player):
                return True, self.current_player

        # 対角線のチェック
        if np.all(np.diag(self.state) == self.current_player):
            return True, self.current_player
        if np.all(np.diag(np.fliplr(self.state)) == self.current_player):
            return True, self.current_player

        # 盤面が埋まっていれば引き分け
        if not any(self.state[i, j] == 0 for i in range(3) for j in range(3)):
            return True, 0  # 引き分け

        return False, None

    def render(self):
        """盤面を●（黒石）、〇（白石）、.（空のマス）で表示する"""
        symbols = {0: "・", 1: "●", 2: "〇"}
        for row in self.state:
            print(" ".join(symbols[cell] for cell in row))
        print("\n")


class RandomAgent:
    """ランダムに手を選ぶエージェント"""
    def choose_action(self, state, available_actions):
        return random.choice(available_actions)


def train_agent():
    """ここでは学習しないランダムエージェントを返す"""
    return RandomAgent()


def play_game(agent):
    env = TicTacToe()
    state = env.reset()
    done = False

    while not done:
        print("現在の盤面:")
        env.render()  # 現在の盤面を表示

        available_actions = env.available_actions()

        if env.current_player == 1:  # 黒 (エージェントの手番)
            action = agent.choose_action(state, available_actions)
        else:  # 白 (人間の手番)
            x, y = map(int, input("石を置く位置（行 列）を入力してください: ").split())
            action = (x, y)

        state, reward, done = env.step(action)

    # ゲーム終了後の盤面を表示
    print("ゲーム終了")
    env.render()

    # 結果を表示
    if reward == 1:
        print("エージェント（黒）の勝ち！")
    elif reward == -1:
        print("プレイヤー（白）の勝ち！")
    else:
        print("引き分け！")


# エージェントの作成
agent = train_agent()

# ゲームをプレイ
play_game(agent)


現在の盤面:
. . .
. . .
. . .


現在の盤面:
. ● .
. . .
. . .


石を置く位置（行 列）を入力してください: 1 1
現在の盤面:
. ● .
. 〇 .
. . .


現在の盤面:
. ● .
● 〇 .
. . .


石を置く位置（行 列）を入力してください: 0 0
現在の盤面:
〇 ● .
● 〇 .
. . .


現在の盤面:
〇 ● .
● 〇 .
. . ●


石を置く位置（行 列）を入力してください: 2 0
現在の盤面:
〇 ● .
● 〇 .
〇 . ●


現在の盤面:
〇 ● .
● 〇 ●
〇 . ●


石を置く位置（行 列）を入力してください: 0 2
ゲーム終了
〇 ● 〇
● 〇 ●
〇 . ●


プレイヤー（白）の勝ち！


In [ ]:
import gym
from gym import spaces
import numpy as np

class TicTacToeEnv(gym.Env):
    def __init__(self, a=1):
        super(TicTacToeEnv, self).__init__()

        # アクションスペースは 0-8 の 9 つのマスに対応
        self.action_space = spaces.Discrete(9)

        # 観測空間は 3**9 までの値（3進数で 9 マスの状態を表現）
        self.observation_space = spaces.Discrete(3**9)

        # 報酬は 0（負け・引き分け）または 1（勝ち）
        self.reward_range = (0, 1)

        # エージェントの石の色 (黒: 1, 白: 2)
        self.agent_color = a
        self.computer_color = 3 - a  # エージェントが黒ならコンピュータは白、逆も然り

        # 盤の状態（0: 空き, 1: 黒, 2: 白）
        self.board = np.zeros(9, dtype=int)

        # ゲーム終了フラグ
        self.done = False

    def reset(self):
        # 盤の初期化
        self.board = np.zeros(9, dtype=int)
        self.done = False

        # エージェントが白ならコンピュータが先攻
        if self.agent_color == 2:
            self._computer_move()

        # 盤の状態（3進数で表現した整数値）と0を返す
        return self._get_observation(), 0

    def step(self, action):
        if self.done:
            raise ValueError("Game is over, please reset the environment.")

        # 無効なアクション（既に石が置かれている場合）
        if self.board[action] != 0:
            return self._get_observation(), 0, True, False, None

        # エージェントが石を置く
        self.board[action] = self.agent_color

        # エージェントが勝ったかチェック
        if self._check_win(self.agent_color):
            self.done = True
            return self._get_observation(), 1, False, True, None

        # 全てのマスが埋まったかどうか
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, False, True, None

        # コンピュータのターン
        self._computer_move()

        # コンピュータが勝ったかチェック
        if self._check_win(self.computer_color):
            self.done = True
            return self._get_observation(), 0, False, True, None

        # 全てのマスが埋まったかどうか
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, False, True, None

        # 終了していないので報酬 0
        return self._get_observation(), 0, False, False, None

    def render(self):
        board_str = ""
        for i in range(9):
            if self.board[i] == 0:
                board_str += "0 "
            elif self.board[i] == 1:
                board_str += "1 "
            else:
                board_str += "2 "
            if (i + 1) % 3 == 0:
                board_str += "\n"
        print(board_str)

    def _get_observation(self):
        # 盤の状態を 3進数として表現
        base3 = np.dot(self.board, [3**i for i in range(9)])
        return int(base3)

    def _computer_move(self):
        # 空いている場所にランダムにコンピュータの石を置く
        available_moves = np.where(self.board == 0)[0]
        if len(available_moves) > 0:
            move = np.random.choice(available_moves)
            self.board[move] = self.computer_color

    def _check_win(self, color):
        # 3つ並びのパターンを定義
        win_patterns = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # 横
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # 縦
            [0, 4, 8], [2, 4, 6]              # 斜め
        ]
        for pattern in win_patterns:
            if all(self.board[i] == color for i in pattern):
                return True
        return False


In [ ]:
import random
import gym
from gym import spaces
import numpy as np

# 3目並べの環境クラス定義
class TicTacToeEnv(gym.Env):
    def __init__(self, a=1):
        super(TicTacToeEnv, self).__init__()

        # アクションスペースは 0-8 の 9 つのマスに対応
        self.action_space = spaces.Discrete(9)

        # 観測空間は 3**9 までの値（3進数で 9 マスの状態を表現）
        self.observation_space = spaces.Discrete(3**9)

        # 報酬は 0（負け・引き分け）または 1（勝ち）
        self.reward_range = (0, 1)

        # エージェントの石の色 (黒: 1, 白: 2)
        self.agent_color = a
        self.computer_color = 3 - a  # エージェントが黒ならコンピュータは白、逆も然り

        # 盤の状態（0: 空き, 1: 黒, 2: 白）
        self.board = np.zeros(9, dtype=int)

        # ゲーム終了フラグ
        self.done = False

    def reset(self):
        # 盤の初期化
        self.board = np.zeros(9, dtype=int)
        self.done = False

        # エージェントが白ならコンピュータが先攻
        if self.agent_color == 2:
            self._computer_move()

        # 盤の状態（3進数で表現した整数値）と0を返す
        return self._get_observation(), 0

    def step(self, action):
        if self.done:
            raise ValueError("Game is over, please reset the environment.")

        # 無効なアクション（既に石が置かれている場合）
        if self.board[action] != 0:
            return self._get_observation(), 0, True, False, None

        # エージェントが石を置く
        self.board[action] = self.agent_color

        # エージェントが勝ったかチェック
        if self._check_win(self.agent_color):
            self.done = True
            return self._get_observation(), 1, False, True, None

        # 全てのマスが埋まったかどうか
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, False, True, None

        # コンピュータのターン
        self._computer_move()

        # コンピュータが勝ったかチェック
        if self._check_win(self.computer_color):
            self.done = True
            return self._get_observation(), 0, False, True, None

        # 全てのマスが埋まったかどうか
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, False, True, None

        # 終了していないので報酬 0
        return self._get_observation(), 0, False, False, None

    def render(self):
        board_str = ""
        for i in range(9):
            if self.board[i] == 0:
                board_str += "0 "
            elif self.board[i] == 1:
                board_str += "1 "
            else:
                board_str += "2 "
            if (i + 1) % 3 == 0:
                board_str += "\n"
        print(board_str)

    def _get_observation(self):
        # 盤の状態を 3進数として表現
        base3 = np.dot(self.board, [3**i for i in range(9)])
        return int(base3)

    def _computer_move(self):
        # 空いている場所にランダムにコンピュータの石を置く
        available_moves = np.where(self.board == 0)[0]
        if len(available_moves) > 0:
            move = np.random.choice(available_moves)
            self.board[move] = self.computer_color

    def _check_win(self, color):
        # 3つ並びのパターンを定義
        win_patterns = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # 横
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # 縦
            [0, 4, 8], [2, 4, 6]              # 斜め
        ]
        for pattern in win_patterns:
            if all(self.board[i] == color for i in pattern):
                return True
        return False

# エージェントとコンピュータの対戦
env = TicTacToeEnv(a=1)  # エージェントが黒（先攻）

# エピソード数
episodes = 10
agent_wins = 0

for episode in range(episodes):
    print(f"Episode {episode + 1}:")

    # 環境をリセット
    observation, _ = env.reset()

    done = False
    while not done:
        # エージェントのターン (ランダムに石を置く位置を選択)
        available_moves = [i for i in range(9) if env.board[i] == 0]
        action = random.choice(available_moves)

        # 行動を環境に渡してステップを実行
        observation, reward, termination, truncation, _ = env.step(action)

        # 勝敗が決まったかチェック
        if termination or truncation:
            env.render()
            if reward == 1:
                print("Agent wins!")
                agent_wins += 1
            else:
                print("Draw or Computer wins!")
            break

# エージェントが勝った回数を表示
print(f"\nAgent won {agent_wins} out of {episodes} episodes.")


Episode 1:
2 0 1 
0 1 2 
1 0 0 

Agent wins!
Episode 2:
1 1 1 
2 1 0 
0 2 2 

Agent wins!
Episode 3:
1 1 2 
1 2 1 
1 2 2 

Agent wins!
Episode 4:
0 1 1 
0 1 2 
2 1 2 

Agent wins!
Episode 5:
1 0 1 
1 0 2 
1 2 2 

Agent wins!
Episode 6:
1 2 0 
1 0 0 
1 2 0 

Agent wins!
Episode 7:
1 0 0 
2 1 2 
0 0 1 

Agent wins!
Episode 8:
2 0 0 
1 1 1 
0 0 2 

Agent wins!
Episode 9:
0 1 1 
1 1 2 
2 2 2 

Draw or Computer wins!
Episode 10:
2 0 2 
2 1 0 
1 1 1 

Agent wins!

Agent won 9 out of 10 episodes.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install stable-baselines3[extra]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00


In [ ]:
import random
import gymnasium as gym  # gym -> gymnasiumに変更
from gymnasium import spaces
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env

# 3目並べの環境クラス定義
class TicTacToeEnv(gym.Env):  # gym.Env -> gymnasium.Envに変更
    def __init__(self, a=1):
        super(TicTacToeEnv, self).__init__()

        # アクションスペースは 0-8 の 9 つのマスに対応
        self.action_space = spaces.Discrete(9)

        # 観測空間は 3**9 までの値（3進数で 9 マスの状態を表現）
        self.observation_space = spaces.Discrete(3**9)

        # 報酬は 0（負け・引き分け）または 1（勝ち）
        self.reward_range = (0, 1)

        # エージェントの石の色 (黒: 1, 白: 2)
        self.agent_color = a
        self.computer_color = 3 - a  # エージェントが黒ならコンピュータは白、逆も然り

        # 盤の状態（0: 空き, 1: 黒, 2: 白）
        self.board = np.zeros(9, dtype=int)

        # ゲーム終了フラグ
        self.done = False

    def reset(self, seed=None, options=None):
        # ランダムシードの設定
        super().reset(seed=seed)

        # 盤の初期化
        self.board = np.zeros(9, dtype=int)
        self.done = False

        # エージェントが白ならコンピュータが先攻
        if self.agent_color == 2:
            self._computer_move()

        # 盤の状態（3進数で表現した整数値）を返す
        return self._get_observation(), {}

    def step(self, action):
        if self.done:
            raise ValueError("Game is over, please reset the environment.")

        # 無効なアクション（既に石が置かれている場合）
        if self.board[action] != 0:
            return self._get_observation(), 0, True, False, {}

        # エージェントが石を置く
        self.board[action] = self.agent_color

        # エージェントが勝ったかチェック
        if self._check_win(self.agent_color):
            self.done = True
            return self._get_observation(), 1, True, False, {}

        # 全てのマスが埋まったかどうか
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, True, False, {}

        # コンピュータのターン
        self._computer_move()

        # コンピュータが勝ったかチェック
        if self._check_win(self.computer_color):
            self.done = True
            return self._get_observation(), 0, True, False, {}

        # 全てのマスが埋まったかどうか
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, True, False, {}

        # 終了していないので報酬 0
        return self._get_observation(), 0, False, False, {}

    def render(self):
        board_str = ""
        for i in range(9):
            if self.board[i] == 0:
                board_str += "0 "
            elif self.board[i] == 1:
                board_str += "1 "
            else:
                board_str += "2 "
            if (i + 1) % 3 == 0:
                board_str += "\n"
        print(board_str)

    def _get_observation(self):
        # 盤の状態を 3進数として表現
        base3 = np.dot(self.board, [3**i for i in range(9)])
        return int(base3)

    def _computer_move(self):
        # 空いている場所にランダムにコンピュータの石を置く
        available_moves = np.where(self.board == 0)[0]
        if len(available_moves) > 0:
            move = np.random.choice(available_moves)
            self.board[move] = self.computer_color

    def _check_win(self, color):
        # 3つ並びのパターンを定義
        win_patterns = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # 横
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # 縦
            [0, 4, 8], [2, 4, 6]              # 斜め
        ]
        for pattern in win_patterns:
            if all(self.board[i] == color for i in pattern):
                return True
        return False

# 環境の作成とチェック
env = TicTacToeEnv(a=1)  # エージェントが黒（先攻）
check_env(env)  # 環境のチェック

# DQNを用いたエージェントの作成
model = DQN("MlpPolicy", env, verbose=1)

# エージェントの訓練
model.learn(total_timesteps=10000)

# エージェントとコンピュータの対戦
episodes = 10
agent_wins = 0

for episode in range(episodes):
    print(f"Episode {episode + 1}:")

    # 環境をリセット
    obs, _ = env.reset()
    done = False
    while not done:
        # DQNエージェントのアクションを決定
        action, _states = model.predict(obs)

        # 行動を環境に渡してステップを実行
        obs, reward, done, _, _ = env.step(action)

        if done:
            env.render()
            if reward == 1:
                print("Agent wins!")
                agent_wins += 1
            else:
                print("Draw or Computer wins!")
            break

# エージェントが勝った回数を表示
print(f"\nAgent won {agent_wins} out of {episodes} episodes.")


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


ストリーミング出力は最後の 5000 行に切り捨てられました。
|    loss             | 0.000433 |
|    n_updates        | 1375     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.24     |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1852     |
|    fps              | 98       |
|    time_elapsed     | 56       |
|    total_timesteps  | 5617     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000825 |
|    n_updates        | 1379     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.26     |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1856     |
|    fps              | 98       |
|    time_elapsed     | 57       |
|    total_timesteps  |

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env

# 三目並べ環境の定義
class TicTacToeEnv(gym.Env):
    def __init__(self, a=1):
        super(TicTacToeEnv, self).__init__()

        # 0から8のマスのアクション
        self.action_space = spaces.Discrete(9)
        # 3進数の観測空間
        self.observation_space = spaces.Discrete(3**9)
        self.reward_range = (0, 1)

        # エージェントの色設定 (1: 黒, 2: 白)
        self.agent_color = a
        self.computer_color = 3 - a  # エージェントが黒ならコンピュータは白

        # 盤面の初期化
        self.board = np.zeros(9, dtype=int)
        self.done = False

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # 盤の初期化
        self.board = np.zeros(9, dtype=int)
        self.done = False

        # エージェントが白ならコンピュータが先攻
        if self.agent_color == 2:
            self._computer_move()

        return self._get_observation(), {}

    def step(self, action):
        if self.done:
            raise ValueError("ゲーム終了。リセットしてください。")

        # 無効なアクション（すでに石が置かれている場所）
        if self.board[action] != 0:
            return self._get_observation(), 0, True, False, {}

        # エージェントの石を配置
        self.board[action] = self.agent_color

        # エージェントが勝ったかの判定
        if self._check_win(self.agent_color):
            self.done = True
            return self._get_observation(), 1, True, False, {}

        # 全てのマスが埋まっているかチェック
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, True, False, {}

        # コンピュータの手番
        self._computer_move()

        # コンピュータが勝ったかの判定
        if self._check_win(self.computer_color):
            self.done = True
            return self._get_observation(), 0, True, False, {}

        # 全てのマスが埋まっているかチェック
        if np.all(self.board != 0):
            self.done = True
            return self._get_observation(), 0, True, False, {}

        # 終了していない場合は報酬0
        return self._get_observation(), 0, False, False, {}

    def render(self):
        """盤面を●（黒石）、〇（白石）、.（空のマス）で表示する"""
        symbols = {0: "・", 1: "●", 2: "〇"}
        reshaped_board = self.board.reshape(3, 3)  # 1次元を3x3に変換
        for row in reshaped_board:
            print(" ".join(symbols[cell] for cell in row))
        print("\n")

    def _get_observation(self):
        # 盤面を3進数で表現
        base3 = np.dot(self.board, [3**i for i in range(9)])
        return int(base3)

    def _computer_move(self):
        available_moves = np.where(self.board == 0)[0]
        if len(available_moves) > 0:
            move = np.random.choice(available_moves)
            self.board[move] = self.computer_color

    def _check_win(self, color):
        # 勝利パターンの定義
        win_patterns = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # 横
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # 縦
            [0, 4, 8], [2, 4, 6]              # 斜め
        ]
        for pattern in win_patterns:
            if all(self.board[i] == color for i in pattern):
                return True
        return False


# 環境の作成とチェック
env = TicTacToeEnv(a=1)  # エージェントは黒（先攻）
check_env(env)  # 環境のチェック

# DQNを使ってエージェントを作成
model = DQN("MlpPolicy", env, verbose=1)

# エージェントの学習
model.learn(total_timesteps=10000)

# エージェントとコンピュータの対戦
episodes = 10
agent_wins = 0

for episode in range(episodes):
    print(f"エピソード {episode + 1}:")

    # 環境をリセット
    obs, _ = env.reset()
    done = False
    step = 0

    while not done:
        print(f"\nターン {step + 1}:")
        env.render()  # 現在の盤面を表示

        # DQNエージェントのアクションを決定
        action, _states = model.predict(obs)
        print(f"エージェントのアクション: {action}")

        # 行動を環境に渡してステップを実行
        obs, reward, done, _, _ = env.step(action)

        # ターン終了後の盤面を表示
        env.render()

        # ターンをカウント
        step += 1

        if done:
            # 最終的な結果を表示
            if reward == 1:
                print("エージェントが勝利しました!")
                agent_wins += 1
            elif reward == 0 and np.all(env.board != 0):
                print("引き分けです!")
            else:
                print("コンピュータが勝利しました!")
            break

# エージェントの勝利回数を表示
print(f"\n10回中、エージェントが勝った回数: {agent_wins}")


ストリーミング出力は最後の 5000 行に切り捨てられました。
|    ep_rew_mean      | 0.05     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1900     |
|    fps              | 104      |
|    time_elapsed     | 55       |
|    total_timesteps  | 5851     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000462 |
|    n_updates        | 1437     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.54     |
|    ep_rew_mean      | 0.06     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1904     |
|    fps              | 104      |
|    time_elapsed     | 55       |
|    total_timesteps  | 5865     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0093   |
|    n_updates        | 1441     |
----------------------------------
-----------------------